In [80]:

"""
SUPER-RSI

The strategy roughly goes like this:

Buy a position when:
    .RSI 1d / 4h / 1h / 30m / 15m <= 30

Close the position when:
    .RSI 1d / 4h / 1h / 30m / 15m >= 70
    .8% fixed stop loss is hit ?? test
"""

import os
from binance.client import Client
import pandas as pd
import datetime
from backtesting import Backtest, Strategy
from backtesting.lib import resample_apply
import sys
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import time

# %%
# Binance API
api_key = os.environ.get('binance_api')
api_secret = os.environ.get('binance_secret')


# %%
client = Client(api_key, api_secret)

# backtest with 6 years of price data 
#-------------------------------------
today = date.today() 
# print(today)
# today - 4 years - 200 days
pastdate = today - relativedelta(years=4) - relativedelta(days=200)

# print(pastdate)
# element = datetime.datetime.strptime(str(pastdate_4years),"%Y-%m-%d")
tuple = pastdate.timetuple()
timestamp = time.mktime(tuple)
# print(timestamp)
# dt_object = datetime.datetime.fromtimestamp(timestamp)
# print(dt_object)

startdate = str(timestamp)
# startdate = "15 Dec, 2018 UTC"
# startdate = "12 May, 2022 UTC"
# startdate = "4 year ago UTC"
# startdate = "10 day ago UTC"
#-------------------------------------


# example when want to choose specific start date
#-------------------------------------
# pastdate = datetime.date(2022, 4, 23)
# pastdate = pastdate - relativedelta(days=200)
# tuple = pastdate.timetuple()
# timestamp = time.mktime(tuple)
# startdate = str(timestamp)
#-------------------------------------


# %%

def EMA(values, n):
    """
    Return exp moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).ewm(span=n, adjust=False).mean()

def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()


def RSI(values, n):
    """Relative strength index"""
    # Approximate; good enough
    gain = pd.Series(values).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)


class System(Strategy):
    rsi_length_1d = 14  # RSI lookback periods
    rsi_length_4h = 14
    rsi_length_1h = 14
    rsi_length_30m = 14
    rsi_length_15m = 14
    rsi_low = 30
    rsi_high = 70
    
    def init(self):
        # Compute moving averages the strategy demands
        # self.ma20 = self.I(SMA, self.data.Close, 20)
        # self.ma50 = self.I(SMA, self.data.Close, 50)
        # self.ma100 = self.I(SMA, self.data.Close, 100)
        
        # Compute RSI
        self.rsi_15m = self.I(RSI, self.data.Close, self.rsi_length_15m)
        self.rsi_30m = resample_apply('30min', RSI, self.data.Close, self.rsi_length_30m)
        self.rsi_1h  = resample_apply('1H', RSI, self.data.Close, self.rsi_length_1h)
        self.rsi_4h  = resample_apply('4H', RSI, self.data.Close, self.rsi_length_4h)
        self.rsi_1d  = resample_apply('D', RSI, self.data.Close, self.rsi_length_1d)
        
        
    def next(self):
        price = self.data.Close[-1]
        # rsi_low = 30
        # rsi_high = 70
        
        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position and
                self.rsi_15m[-1] <= self.rsi_low and
                self.rsi_30m[-1] <= self.rsi_low and
                self.rsi_1h[-1]  <= self.rsi_low and
                self.rsi_4h[-1]  <= self.rsi_low and
                self.rsi_1d[-1]  <= self.rsi_low):
            self.buy()
        
        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        else: 
            if (self.rsi_15m[-1] >= self.rsi_high and
                self.rsi_30m[-1] >= self.rsi_high and
                self.rsi_1h[-1]  >= self.rsi_high and
                self.rsi_4h[-1]  >= self.rsi_high and
                self.rsi_1d[-1]  >= self.rsi_high): 
                self.position.close()
            

# %%
def get_data(Symbol, time_frame, start_date):
    frame = pd.DataFrame(client.get_historical_klines(Symbol,
                                                      time_frame,
                                                      start_date
                                                      ))
    
    frame = frame.iloc[:,:6] # use the first 5 columns
    frame.columns = ['Time','Open','High','Low','Close','Volume'] #rename columns
    frame[['Open','High','Low','Close','Volume']] = frame[['Open','High','Low','Close','Volume']].astype(float) #cast to float
    frame['Date'] = frame['Time'].astype(str) 
    # set the 'date' column as the DataFrame index
    frame.set_index(pd.to_datetime(frame['Date'], unit='ms'), inplace=True) #make human readable timestamp)
    frame = frame.drop(['Date'], axis=1)
    # frame.index = [dt.datetime.fromtimestamp(x/1000.0) for x in frame.Time]


    # format = '%Y-%m-%d %H:%M:%S'
    # frame['Time'] = pd.to_datetime(frame['Time'], format=format)
    
    # frame = frame.set_index(pd.DatetimeIndex(frame['Time']))
    # frame = frame.drop(['Time'], axis=1)
    return frame



In [81]:
symbol = "BTCUSDT"
time_frame = client.KLINE_INTERVAL_15MINUTE
start_date = startdate
df = get_data(symbol, time_frame, start_date)
df

,Time,Open,High,Low,Close,Volume
Date,,,,,,
2018-08-18 00:00:00,1534550400000,6579.04,6584.49,6541.89,6551.72,446.699894
2018-08-18 00:15:00,1534551300000,6554.15,6575.00,6545.00,6569.99,641.722886
2018-08-18 00:30:00,1534552200000,6570.00,6599.96,6569.02,6598.96,826.299474
2018-08-18 00:45:00,1534553100000,6599.07,6620.00,6568.99,6603.02,898.345913
2018-08-18 01:00:00,1534554000000,6603.94,6616.53,6565.00,6568.00,748.455856
...,...,...,...,...,...,...
2023-03-06 15:15:00,1678115700000,22457.13,22484.66,22432.07,22439.26,3001.572270
2023-03-06 15:30:00,1678116600000,22439.26,22474.96,22433.50,22472.54,1921.528850
2023-03-06 15:45:00,1678117500000,22472.54,22602.19,22470.51,22553.42,7999.265990


In [57]:
df_30m = df.resample('30min').last()
df_30m
df_1h = df.resample('1H').last()
df_1h
df_4h = df.resample('4H').last()
df_4h
df_1d = df.resample('1D').last()
df_1d

,Time,Open,High,Low,Close,Volume
Date,,,,,,
2023-02-24 06:00:00,1.677219e+12,23849.020,23887.055,23781.770,23834.970,3343.514940
2023-02-24 06:30:00,1.677221e+12,23861.100,23904.395,23847.105,23872.430,1967.831060
2023-02-24 07:00:00,1.677222e+12,23885.155,23892.680,23808.995,23837.605,2810.056490
2023-02-24 07:30:00,1.677224e+12,23819.225,23858.610,23797.795,23837.130,2780.816655
2023-02-24 08:00:00,1.677226e+12,23797.105,23854.150,23757.020,23806.380,3123.485550
...,...,...,...,...,...,...
2023-03-06 13:30:00,1.678110e+12,22430.705,22473.055,22410.310,22446.430,3501.828335
2023-03-06 14:00:00,1.678112e+12,22428.605,22443.655,22409.325,22427.730,1757.831380
2023-03-06 14:30:00,1.678113e+12,22439.255,22467.105,22411.735,22444.785,3926.551035


In [82]:
%%time
bt = Backtest(df, System, cash=100000, commission=0.001)
stats = bt.run()
print(stats)

Start                     2018-08-18 00:00:00
End                       2023-03-06 16:15:00
Duration                   1661 days 16:15:00
Exposure Time [%]                   47.609208
Equity Final [$]                 182353.36663
Equity Peak [$]                  484034.66663
Return [%]                          82.353367
Buy & Hold Return [%]              244.011191
Return (Ann.) [%]                   14.103919
Volatility (Ann.) [%]               56.556611
Sharpe Ratio                         0.249377
Sortino Ratio                        0.454851
Calmar Ratio                         0.210256
Max. Drawdown [%]                  -67.079823
Avg. Drawdown [%]                   -2.697545
Max. Drawdown Duration      479 days 11:15:00
Avg. Drawdown Duration        5 days 21:53:00
# Trades                                    5
Win Rate [%]                             60.0
Best Trade [%]                     105.709095
Worst Trade [%]                    -47.956146
Avg. Trade [%]                    

In [83]:
bt.plot()

/home/joaosilva/.local/lib/python3.10/site-packages/backtesting/_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '4H'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "
/home/joaosilva/.local/lib/python3.10/site-packages/backtesting/_plotting.py:148: FutureWarning: Passing method to Int64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  new_bar_idx = new_index.get_loc(mean_time, method='nearest')
/home/joaosilva/.local/lib/python3.10/site-packages/backtesting/_plotting.py:148: FutureWarning: Passing method to Int64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  new_bar_idx = new_index.get_loc(mean_time, method='nearest')
/home/joaosilva/.local/lib/python3.10/site-packages/backtesting/_plotting.py:148: FutureWarning: Passing method to In

Row(id='3692', ...)

Gtk-Message: 16:25:55.099: Failed to load module "canberra-gtk-module"
Gtk-Message: 16:25:55.102: Failed to load module "canberra-gtk-module"
Gtk-Message: 16:25:56.781: Failed to load module "canberra-gtk-module"
Gtk-Message: 16:25:56.783: Failed to load module "canberra-gtk-module"
[57:57:0306/162556.838132:ERROR:angle_platform_impl.cc(43)] Display.cpp:1014 (initialize): ANGLE Display::initialize error 12289: GLX is not present.
ERR: Display.cpp:1014 (initialize): ANGLE Display::initialize error 12289: GLX is not present.
[57:57:0306/162556.838295:ERROR:gl_display.cc(504)] EGL Driver message (Critical) eglInitialize: GLX is not present.
[57:57:0306/162556.838362:ERROR:gl_display.cc(917)] eglInitialize OpenGL failed with error EGL_NOT_INITIALIZED, trying next display type
[57:57:0306/162556.838520:ERROR:angle_platform_impl.cc(43)] Display.cpp:1014 (initialize): ANGLE Display::initialize error 12289: GLX is not present.
ERR: Display.cpp:1014 (initialize): ANGLE Display::initialize error

Opening in existing browser session.


In [84]:
# bt.optimize(d_rsi=range(10, 35, 5),
#                   w_rsi=range(10, 35, 5),
#                   level=range(30, 80, 10))

bt.optimize(rsi_length_1d = range(10, 35, 5),  # RSI lookback periods
            rsi_length_4h = range(10, 35, 5),
            rsi_length_1h = range(10, 35, 5),
            rsi_length_30m = range(10, 35, 5),
            rsi_length_15m = range(10, 35, 5),
            rsi_low = range(10, 30, 10),
            rsi_high = range(70, 100, 10))

/home/joaosilva/.local/lib/python3.10/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 18750 configurations.
  output = _optimize_grid()


In [ ]:


    bt.plot()
    

    stats, heatmap = bt.optimize(
    n1=range(5, 100, 5),
    n2=range(10, 100, 5),
    constraint=lambda param: param.n1 < param.n2,
    maximize='Equity Final [$]',
    return_heatmap=True
    )

    dfbema = pd.DataFrame(heatmap.sort_values().iloc[-1:])
    n1 = dfbema.index.get_level_values(0)[0]
    n2 = dfbema.index.get_level_values(1)[0]
    returnPerc = round(stats['Return [%]'],2)
    BuyHoldReturnPerc = round(stats['Buy & Hold Return [%]'],2)
    BacktestStartDate = str(df.index[0])
    num_trades = stats['# Trades']

    # lista
    print("n1=",n1)
    print("n2=",n2)
    print("Return [%] = ",round(returnPerc,2))
    print("Buy & Hold Return [%] = ",round(BuyHoldReturnPerc,2))
    print("Backtest start date =", BacktestStartDate)
    print("Trades =", num_trades)